In [ ]:
# En caso de no tenerlo
# % pip install mlflow

In [1]:
import mlflow
import mlflow.pyfunc
import pandas as pd
import joblib

Dentro de variables `model_name` y `model_version` se guardan como estan etiquedos en MLFLow.

In [ ]:
! export MLFLOW_TRACKING_URI="https://dagshub.com/empleo-inducido/project-ml.mlflow"
! export MODEL_NAME="testing-model"
! export MODEL_VERSION=1

# Construir la variable MODEL_URI
! export MODEL_URI="models:/${MODEL_NAME}/${MODEL_VERSION}"

# Ejecutar el comando mlflow models serve
! mlflow models serve -m "$MODEL_URI" -p 5000

In [4]:
df = pd.read_csv('/home/project/data/simulados/simulados.csv')
df = df.drop(['TIPO_PACIENTE',], axis=1)
scaler = joblib.load('/home/project/models/preprocess_utils/scaler.pkl')
encoder = joblib.load('/home/project/models/preprocess_utils/encoder.pkl')
encoded_columns  = encoder.transform(df[['SEXO', 'ESTATUS_CASO', 'RESULTADO_PCR']])
scaled_columns = scaler.fit_transform(df.drop(['SEXO', 'ESTATUS_CASO', 'RESULTADO_PCR'], axis=1))
encoded_scaled_df = pd.DataFrame(encoded_columns, columns=encoder.get_feature_names_out(['SEXO', 'ESTATUS_CASO', 'RESULTADO_PCR']))
scaled_df = pd.DataFrame(scaled_columns, columns=df.drop(['SEXO', 'ESTATUS_CASO', 'RESULTADO_PCR'], axis=1).columns)
data = pd.concat([encoded_scaled_df, scaled_df], axis=1)

In [5]:
import requests
import pandas as pd
import json

In [30]:
data_list = data.to_numpy().tolist()

In [31]:
data_json = json.dumps(data_list)

In [32]:
url = 'http://localhost:5000/invocations'
headers = {'Content-Type': 'application/json'}

In [33]:
response = requests.post(url, data=data_json, headers=headers)

# Imprimir la respuesta
print(response.json())

{'error_code': 'BAD_REQUEST', 'message': "Invalid input. The input must be a JSON dictionary with exactly one of the input fields {'inputs', 'instances', 'dataframe_split', 'dataframe_records'}. Received a list."}
